# GIS 공간 정보 데이터

- 좌표계: GRS80 UTM-K / EPSG:5179
- 출처: <http://www.gisdeveloper.co.kr/?p=2332>

---

In [1]:
import pandas as pd
import geopandas as gpd

---

# 1. 시도 좌표계

In [2]:
ctp = gpd.read_file('GIS/CTPRVN_202302/ctp_rvn.shp', encoding='cp949')
ctp.head()

,CTPRVN_CD,CTP_ENG_NM,CTP_KOR_NM,geometry
0,11,Seoul,서울특별시,"POLYGON ((966987.226 1941110.946, 966987.119 1..."
1,26,Busan,부산광역시,"MULTIPOLYGON (((1148194.981 1685460.055, 11481..."
2,27,Daegu,대구광역시,"POLYGON ((1087859.999 1760097.461, 1087859.951..."
3,28,Incheon,인천광역시,"MULTIPOLYGON (((847834.755 1881816.658, 847840..."
4,29,Gwangju,광주광역시,"POLYGON ((932712.687 1696168.692, 932781.680 1..."


---

# 2. 시군구 좌표계

In [3]:
sig = gpd.read_file('GIS/SIG_202302/sig.shp', encoding='cp949')
sig.head()

,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,geometry
0,11110,Jongno-gu,종로구,"POLYGON ((956615.453 1953567.199, 956621.579 1..."
1,11140,Jung-gu,중구,"POLYGON ((957890.386 1952616.746, 957909.908 1..."
2,11170,Yongsan-gu,용산구,"POLYGON ((953115.761 1950834.084, 953114.206 1..."
3,11200,Seongdong-gu,성동구,"POLYGON ((959681.109 1952649.605, 959842.412 1..."
4,11215,Gwangjin-gu,광진구,"POLYGON ((964825.058 1952633.250, 964875.565 1..."


---

# 3. 읍면동 좌표계

In [4]:
emd = gpd.read_file('GIS/EMD_202302/emd.shp', encoding='cp949')
emd.head()

,EMD_CD,EMD_ENG_NM,EMD_KOR_NM,geometry
0,11110101,Cheongun-dong,청운동,"POLYGON ((953700.022 1954605.065, 953693.871 1..."
1,11110102,Singyo-dong,신교동,"POLYGON ((953233.465 1953996.984, 953235.183 1..."
2,11110103,Gungjeong-dong,궁정동,"POLYGON ((953560.228 1954257.466, 953561.190 1..."
3,11110104,Hyoja-dong,효자동,"POLYGON ((953519.843 1953890.785, 953518.489 1..."
4,11110105,Changseong-dong,창성동,"POLYGON ((953516.123 1953734.362, 953516.526 1..."


---

# 4. 시군구 master 생성

## 1) sig-master.csv

In [5]:
# 서울만 추출
sig_master = sig.loc[sig['SIG_CD'].str[:2] == '11']
sig_master = sig_master.loc[:, ['SIG_CD', 'SIG_KOR_NM']]
sig_master.columns = ['sig_cd', 'sig_nm']
sig_master.head()

,sig_cd,sig_nm
0,11110,종로구
1,11140,중구
2,11170,용산구
3,11200,성동구
4,11215,광진구


In [6]:
sig_master.to_csv('master/sig-master.csv', index=False)

## 2) sig-geo-master.shp

In [7]:
# 서울만 추출
sig_geo_master = sig.loc[sig['SIG_CD'].str[:2] == '11']
sig_geo_master = sig_geo_master.loc[:, ['SIG_CD', 'SIG_KOR_NM', 'geometry']]
sig_geo_master.columns = ['sig_cd', 'sig_nm', 'geometry']
sig_geo_master.head()

,sig_cd,sig_nm,geometry
0,11110,종로구,"POLYGON ((956615.453 1953567.199, 956621.579 1..."
1,11140,중구,"POLYGON ((957890.386 1952616.746, 957909.908 1..."
2,11170,용산구,"POLYGON ((953115.761 1950834.084, 953114.206 1..."
3,11200,성동구,"POLYGON ((959681.109 1952649.605, 959842.412 1..."
4,11215,광진구,"POLYGON ((964825.058 1952633.250, 964875.565 1..."


In [8]:
sig_geo_master.to_file('master/sig-geo-master.shp', index=False, encoding='cp949')

## 3) emd-master.csv

In [9]:
# 서울만 추출
emd_master = emd.loc[emd['EMD_CD'].str[:2] == '11']

# 자치구 컬럼 생성
emd_master['sig_cd'] = emd_master['EMD_CD'].str[:5]
emd_master = pd.merge(emd_master, sig_master, on='sig_cd')

# 컬럼명 정리
emd_master.rename(columns={'EMD_CD':'emd_cd', 'EMD_KOR_NM':'emd_nm'}, inplace=True)
emd_master = emd_master.loc[:, ['sig_cd', 'sig_nm', 'emd_cd', 'emd_nm']]
emd_master.head()

C:\Users\User\anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,sig_cd,sig_nm,emd_cd,emd_nm
0,11110,종로구,11110101,청운동
1,11110,종로구,11110102,신교동
2,11110,종로구,11110103,궁정동
3,11110,종로구,11110104,효자동
4,11110,종로구,11110105,창성동


In [10]:
emd_master.to_csv('master/emd-master.csv', index=False)

## 4) emd-geo-master.shp

In [11]:
# 서울만 추출
emd_geo_master = emd.loc[emd['EMD_CD'].str[:2] == '11']

# 자치구 컬럼 생성
emd_geo_master['sig_cd'] = emd_geo_master['EMD_CD'].str[:5]
emd_geo_master = pd.merge(emd_geo_master, sig_master, on='sig_cd')

# 컬럼명 정리
emd_geo_master.rename(columns={'EMD_CD':'emd_cd', 'EMD_KOR_NM':'emd_nm'}, inplace=True)
emd_geo_master = emd_geo_master.loc[:, ['sig_cd', 'sig_nm', 'emd_cd', 'emd_nm', 'geometry']]
emd_geo_master.head()

C:\Users\User\anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,sig_cd,sig_nm,emd_cd,emd_nm,geometry
0,11110,종로구,11110101,청운동,"POLYGON ((953700.022 1954605.065, 953693.871 1..."
1,11110,종로구,11110102,신교동,"POLYGON ((953233.465 1953996.984, 953235.183 1..."
2,11110,종로구,11110103,궁정동,"POLYGON ((953560.228 1954257.466, 953561.190 1..."
3,11110,종로구,11110104,효자동,"POLYGON ((953519.843 1953890.785, 953518.489 1..."
4,11110,종로구,11110105,창성동,"POLYGON ((953516.123 1953734.362, 953516.526 1..."


In [13]:
emd_geo_master.to_file('master/emd-geo-master.shp', index=False, encoding='cp949')